In [2]:
import numpy as np

# build examples
dataWidth = 6
minSlope = -3
maxSlope = 3
i=0
#dataset = np.array()
dataset = np.empty(dataWidth)
slope = np.empty(100)

# build varying slopes
for currentSlope in range (minSlope, maxSlope+1):
    print("currentSlope: ", currentSlope)
    slope[i] = currentSlope
    lastValue = 0
    if (currentSlope == 0):
        newdata = np.zeros(dataWidth)
    else:
        newdata = np.arange(0, dataWidth*currentSlope, currentSlope)
    dataset = np.vstack((dataset, newdata))
    i+=1
    
print(dataset)

currentSlope:  -3
currentSlope:  -2
currentSlope:  -1
currentSlope:  0
currentSlope:  1
currentSlope:  2
currentSlope:  3
[[  0.   0.   0.   0.   0.   0.]
 [  0.  -3.  -6.  -9. -12. -15.]
 [  0.  -2.  -4.  -6.  -8. -10.]
 [  0.  -1.  -2.  -3.  -4.  -5.]
 [  0.   0.   0.   0.   0.   0.]
 [  0.   1.   2.   3.   4.   5.]
 [  0.   2.   4.   6.   8.  10.]
 [  0.   3.   6.   9.  12.  15.]]
